In [ ]:
import matplotlib.pyplot as plt
import pickle
import os

from dotenv import load_dotenv
from importlib import reload

from src import strava_api
from src import strava_cleaner
from src import strava_visualizer
reload(strava_api)
reload(strava_cleaner)
reload(strava_visualizer)

from src.strava_api import StravaClient
from src.strava_cleaner import StravaCleaner
from src.strava_visualizer import StravaVisualizer

load_dotenv()

plt.rc("xtick", labelsize=22)
plt.rc("ytick", labelsize=22)
plt.style.use("ggplot")

In [ ]:
client = StravaClient(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    authorization_code=os.getenv("AUTHORIZATION_CODE"),
    refresh_token=os.getenv("REFRESH_TOKEN")
)

cleaner = StravaCleaner()

visualizer = StravaVisualizer(api_key=os.getenv("GEOAPIFY_API_KEY"))

## Extract Strava Activities

In [ ]:
activities = client.extract_activities(file=None)

In [ ]:
# with open("../data/raw/activities.pkl", "rb") as f:
#     activities = pickle.load(f)

## Clean Strava Activities

In [ ]:
activities_clean = cleaner.clean_activities(activities=activities, file="../data/clean/activities.pkl", progress_bar=False)

## Extract Strava Activity Streams

In [ ]:
activity_streams_all = []

In [ ]:
# Run this cell as needed and append into activity_streams_all
# Necessary to get around Strava's 200/15min API call limit
# If you have fewer than 200 tracks, you can specify the file here and save it that way
activity_streams = client.extract_activity_streams(
    activity_ids=activities_clean["activity_id"].values[0:150],
    file=None
)

activity_streams_all.extend(activity_streams)

In [ ]:
# with open("../data/raw/streams.pkl", "rb") as f:
#     activity_streams_all = pickle.load(f)

## Clean Strava Activity Streams

In [ ]:
streams = cleaner.clean_activity_streams(activity_streams_all, progress_bar=True, file="../data/clean/activity_streams.pkl")

## Export GPX files from Strava Activity Streams

In [ ]:
streams_clean = cleaner.create_gpx_files(
    activities=activities_clean,
    activity_streams=streams,
    save_dir="../data/gpx/"
)

## Generate Heatmap

In [ ]:
heatmap = visualizer.generate_heatmap(
    gpx_dir="../data/gpx/",
    file="../img/heatmap.png",
    zoom=12,  # Zoom level for the tiles. -1 auto zooms
    sigma=1,  # Width of heatmap track
    year_filter=None,  # Activities for this year
    lat_lon_bounds=(39.813811, -105.558014, 40.166281, -105.195465)  # Boulder
    # lat_lon_bounds=(37.788624, -122.392159, 37.895718, -122.219810)  # Oakland
)

In [ ]:
# Some open source map tiles cause problems unless you visit the webpage
# Displaying and visiting the urls sometimes fixes the issue
# Looking at you Wikimedia
for url in visualizer.bad_urls:
    print(url)